In [1]:
#https://keras.io/examples/generative/wgan_gp/
#https://keras.io/examples/generative/wgan-graphs/

In [2]:
import os,shutil,random
from sklearn.preprocessing import MinMaxScaler
import joblib
from IPython.display import clear_output
import pandas as pd
import h5py
import tensorflow as tf
from IPython.display import clear_output,display, HTML
from sklearn.model_selection import train_test_split
import datetime as dt
import numpy as np
from tensorboard.plugins.hparams import api as hp
from src.wgangp.utils import Scaler
%load_ext tensorboard

2024-08-16 09:04:00.316102: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-16 09:04:00.481420: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-16 09:04:00.614711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 09:04:00.703450: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 09:04:00.727850: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 09:04:00.948611: I tensorflow/core/platform/cpu_feature_gu

In [3]:
#================== initialization ==================
#best LR = 1e-4,EPOCHS = 10,"discriminator_extra_steps":1
#best LR = 1e-4,EPOCHS = 15,"discriminator_extra_steps":1

currentTM=dt.datetime.now().strftime("%Y-%m-%dT%H%M%S")
PROJECT = "wgangpModel"
LATENT_DIM = 16
LR = 1e-4
EPOCHS = 10
BATCH_SIZE = 32

PARQUET_PATH = './data/OptionsEOD_STG.parquet'

UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
SCALER_COL  = ['UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE','INTRINSIC_VALUE','DTE','TOTAL_VOLUME','C_VEGA','P_VEGA',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]

MODEL_PATH = "./models/"
H5_PATH = './data/OptTrainData/'
STACK_DATA_SHAPE = np.empty((0,) + (16, len( SCALER_COL)  ) ) 

WANDB_LOG = True
RESUME = False
SUMMARY = True
log_dir = f"/app/logs/{PROJECT}/"+dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# Scaler = joblib.load(SCALER_PATH )
# Scaler_Price = joblib.load(SCALER_PRICE_PATH )

#DISPLAY = ['map','summary',None]
DISPLAY = 'summary'


In [4]:
###===== load Scaler
QUOTE_COL = ['C_BID',	'C_ASK',  'P_BID',	'P_ASK']
VEGA_COL =  ["C_VEGA","P_VEGA"] 
VOLUME_COL =  ["TOTAL_VOLUME","C_VOLUME","P_VOLUME"] 

SCALER_QUOTE_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in QUOTE_COL]
SCALER_VEGA_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in VEGA_COL]
SCALER_VOLUME_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v in VOLUME_COL]
SCALER_OTHER_COL_INDEX = [i for i,v in enumerate(SCALER_COL) if v not in QUOTE_COL+VEGA_COL+VOLUME_COL ]

SCALER = Scaler(
    [v for v in SCALER_COL if v not in QUOTE_COL+VEGA_COL+VOLUME_COL ] + ['QUOTE','VEGA','VOLUME']
    , path="./data/scaler/")

[Load-UNDERLYING_LAST] :  ./data/scaler/scaler_UNDERLYING_LAST.gz
[Load-STRIKE] :  ./data/scaler/scaler_STRIKE.gz
[Load-STRIKE] :  ./data/scaler/scaler_STRIKE_DISTANCE.gz
[Load-STRIKE_DISTANCE] :  ./data/scaler/scaler_STRIKE_DISTANCE.gz
[Load-INTRINSIC_VALUE] :  ./data/scaler/scaler_INTRINSIC_VALUE.gz
[Load-DTE] :  ./data/scaler/scaler_DTE.gz
[Load-QUOTE] :  ./data/scaler/scaler_QUOTE.gz
[Load-VEGA] :  ./data/scaler/scaler_VEGA.gz
[Load-VOLUME] :  ./data/scaler/scaler_VOLUME.gz


In [5]:
##['UNDERLYING_LAST','STRIKE','STRIKE_DISTANCE','INTRINSIC_VALUE','DTE','TOTAL_VOLUME','C_VEGA','P_VEGA',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
select_x = [i for i,c in  enumerate(SCALER_COL) if c in ['DTE','INTRINSIC_VALUE','C_VEGA','P_VEGA'] ]
select_y = [i for i,c in enumerate(SCALER_COL) if c in ['C_BID','C_ASK',  'P_BID',	'P_ASK'] ]

In [6]:
import wandb
from wandb.integration.keras import WandbCallback
CONFIG = {
          "learning_rate": LR,
          "epochs": EPOCHS,
          "batch_size": BATCH_SIZE,
          "architecture": "wgangp",
          "dataset": "OptionsChaine",
          "generator_dense_units":[128,256,512,256,128],
          "generator_dropout_rate":0.2,
          "discriminator_dense_units":[64,128,256,512],
          "discriminator_dropout_rate":0.2,
    
          "discriminator_use_bias":False,
          "discriminator_use_dropout":True,
          "discriminator_use_bn":False,
    
          "generator_use_bias":False,
          "generator_use_dropout":True,
          "generator_use_bn":False,
    
          "transform":True,
          "discriminator_extra_steps":1,
          "x_col":select_x,
          "y_col":select_y,
          "gp_weight":(10,),
          "gp_cap": None
           }



notes = f"""
DenseLayer model
additional_con_loss_p_1/c_1 * 0.01 
discriminator_extra_steps : {CONFIG['discriminator_extra_steps']}
gp_weight:{CONFIG['gp_weight']}
LR:{LR}
gp cap : {CONFIG['gp_cap']}
reduce_mean(ASK-BID) 
"""


if WANDB_LOG :
    wandb.login()
    run = wandb.init(project=PROJECT, 
                     name=currentTM, 
                     config=CONFIG,
                     notes=notes
                    )

wandb: Currently logged in as: wasan-sinlapa. Use `wandb login --relogin` to force relogin


In [7]:
#=====================================================================

In [8]:
from src.wgangp.modelNN import OptionChainGenerator
from src.wgangp.layers import generatorDense as generator
from src.wgangp.layers import discriminatorDense as discriminator

In [9]:


model = OptionChainGenerator(
    discriminator = discriminator(
            input_shape= (16,len(select_y) ), 
            dense_units = CONFIG["discriminator_dense_units"], 
            dropout_rate= CONFIG["discriminator_dropout_rate"],
            use_bias=CONFIG["discriminator_use_bias"],
            use_dropout=CONFIG["discriminator_use_dropout"],
            use_bn=CONFIG["discriminator_use_bn"]
           ), 
    generator = generator(
            input_dim = (16,len(select_y) ),
            output_dim = (16,len(select_x) ) ,
            dense_units = CONFIG["generator_dense_units"],
            dropout_rate= CONFIG["generator_dropout_rate"],
            use_bias=CONFIG["generator_use_bias"],
            use_dropout=CONFIG["generator_use_dropout"],
            use_bn=CONFIG["generator_use_bn"]
           ),
    discriminator_extra_steps = CONFIG["discriminator_extra_steps"],
    output_col=[SCALER_COL[i] for i in select_y] ,
    scaler = SCALER,
    gp_weight=CONFIG["gp_weight"],
    gp_cap=CONFIG["gp_cap"]
)


model.compile(
    d_optimizer = tf.optimizers.Adam(
    learning_rate=LR, beta_1=0.5, beta_2=0.9, clipvalue=10
    ),
    g_optimizer = tf.optimizers.Adam(
    learning_rate=LR, beta_1=0.5, beta_2=0.9, clipvalue=10
    )
)

In [10]:
################## show model ######################
if DISPLAY == 'map' :
    from tensorflow.keras.utils import model_to_dot
    from IPython.display import SVG, display
    
    def display_model(model, width=1024, height=512):
        dot = model_to_dot(model, show_shapes=True, show_layer_names=True)
        svg_data = dot.create(prog='dot', format='svg').decode("utf-8")
        svg_html = f'<div style="width:{width}px;height:{height}px;">{svg_data}</div>'
        display(HTML(svg_html))

In [11]:
## Example usage:
## Display the generator model with reduced size
if DISPLAY == 'map' :
    display_model(model.generator, width, height=512)
if DISPLAY == 'summary' :
    model.generator.summary()

Model: "generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_data (InputLayer)         │ (32, 16, 4)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (32, 64)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (32, 64)               │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (32, 64)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense0_128 (Dense)              │ (32, 128)              │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout0_128 (Dropout)          │ (32, 128)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense1_256 (Dense)              │ (32, 256)              │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout1_256 (Dropout)          │ (32, 256)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense2_512 (Dense)              │ (32, 512)              │       131,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout2_512 (Dropout)          │ (32, 512)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense3_256 (Dense)              │ (32, 256)              │       131,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout3_256 (Dropout)          │ (32, 256)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense4_128 (Dense)              │ (32, 128)              │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout4_128 (Dropout)          │ (32, 128)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 64)               │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Reshape-output (Reshape)        │ (32, 16, 4)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 344,384 (1.31 MB)

 Trainable params: 344,256 (1.31 MB)

 Non-trainable params: 128 (512.00 B)

In [12]:
if DISPLAY == 'map' :
    display_model(model.discriminator, width=2500, height=512)
if DISPLAY == 'summary' :
    model.discriminator.summary()

Model: "discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_y_data (InputLayer)       │ (32, 16, 4)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (32, 64)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense0_64 (Dense)               │ (32, 64)               │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout0_64 (Dropout)           │ (32, 64)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense1_128 (Dense)              │ (32, 128)              │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout1_128 (Dropout)          │ (32, 128)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense2_256 (Dense)              │ (32, 256)              │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout2_256 (Dropout)          │ (32, 256)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense3_512 (Dense)              │ (32, 512)              │       131,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout3_512 (Dropout)          │ (32, 512)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ logic (Dense)                   │ (32, 1)                │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 176,641 (690.00 KB)

 Trainable params: 176,641 (690.00 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
#================== loadmodel ====================

In [14]:
from tensorflow.keras.models import load_model
model_path = MODEL_PATH+f'{PROJECT}'
if not RESUME :
    if os.path.exists(model_path) :
        shutil.rmtree(model_path)
if not os.path.exists(model_path):
    os.makedirs(model_path)
    model.generator.save(model_path+f'/'+f'generator.keras') 
    model.discriminator.save(model_path+f'/'+f'discriminator.keras') 
else:
    model.generator = load_model(model_path+'/'+f'generator.keras') 
    model.discriminator = load_model(model_path+'/'+f'discriminator.keras') 

In [15]:
# ================== train model ==================
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
#SET MODEL VAR
STACK_DATA = STACK_DATA_SHAPE 
#INIT MODEL VAR
STOP_MODEL = False

#set PartitionDate[:] for limit range
for partdate in PartitionDate[:] :
    clear_output(wait=False)
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
    data_shape = DATA.shape
    ###transform
    if CONFIG['transform'] :
        #other index
        for i, c in enumerate (SCALER_COL):
            DATA[:,:,[i]] = SCALER.groupTransform(c,DATA[:,:,[i]].reshape(-1,1)
                                                ,QUOTE_COL = QUOTE_COL
                                               ,VEGA_COL = VEGA_COL
                                               ,VOLUME_COL = VOLUME_COL
                                                 ).reshape(-1, 16, 1)
    DATA = np.vstack((DATA ,STACK_DATA))

    if len(DATA) < 64 :
        #stack data
        STACK_DATA = np.vstack((STACK_DATA ,DATA))
    else: 
        STACK_DATA = np.empty((0,) + data_shape[1:] )
        X = DATA[:, :, select_x]  # เลือกข้อมูล select_x สำหรับ X
        Y = DATA[:, :, select_y]  # เลือกข้อมูล select_y เสำหรับ Y
        x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
        random.shuffle(PartitionDate)
        tf.keras.backend.clear_session() 
        history = model.fit(x_train , y_train, epochs=CONFIG['epochs'], batch_size=BATCH_SIZE, validation_data=(x_val, y_val) ,callbacks=[tensorboard_callback])
        if  np.isnan(  np.average( history.history['generator_loss'] )  ) or np.isnan(  np.average( history.history['discriminator_loss'] )  ):
            STOP_MODEL = True 
    
        if WANDB_LOG :
            LogKeys = history.history.keys()
            LogVal={}
            for k in LogKeys:  
                LogVal[k] = np.average(  history.history[k] )
            wandb.log(LogVal, commit=True)
    if STOP_MODEL :
        break
    
            
    model.generator.save(model_path+f'/'+f'generator.keras') 
    model.discriminator.save(model_path+f'/'+f'discriminator.keras') 
if WANDB_LOG : wandb.finish()

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - diff_logits: 0.0000e+00 - discriminator_loss: 0.0000e+00 - fake_logits: 0.0000e+00 - generator_loss: 0.0448 - generator_loss_c1: 7.1047e-04 - generator_loss_c2: 7.0568e-04 - generator_loss_p1: 4.7767e-04 - generator_loss_p2: 4.5399e-04 - generator_loss_wgan: 0.1136 - gradient_penalty: 0.0000e+00 - real_logits: 0.0000e+00 - val_discriminator_loss: 0.0000e+00 - val_generator_loss: 0.0145
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - diff_logits: 0.0000e+00 - discriminator_loss: 0.0000e+00 - fake_logits: 0.0000e+00 - generator_loss: 0.0359 - generator_loss_c1: 7.1038e-04 - generator_loss_c2: 7.0562e-04 - generator_loss_p1: 4.7755e-04 - generator_loss_p2: 4.5387e-04 - generator_loss_wgan: 0.1136 - gradient_penalty: 0.0000e+00 - real_logits: 0.0000e+00 - val_discriminator_loss: 0.0000e+00 - val_generator_loss: 0.0146
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - diff_logits: 0.0000e+00 - discriminator_loss: 0.0000e+00 - fake

diff_logits,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
discriminator_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
fake_logits,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
generator_loss,▃▂▂▁▁▂▁▁▁▂▁▁▅▁▄▁▁▁▃▁▁▂▁▅▁▃▁█▁▁▂▁▁▁▁▁▄▁▂▁
generator_loss_c1,█▇▅▄▄█▇▆▆▅▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
generator_loss_c2,▇▇▅▄▄█▇▆▆▅▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
generator_loss_p1,██▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
generator_loss_p2,██▆▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
generator_loss_wgan,█▅▃▂▂▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
gradient_penalty,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
real_logits,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [16]:
`====================== GET ZERO Val. ========================

SyntaxError: invalid syntax (424402459.py, line 1)

In [ ]:
D_ZERO = np.full((1,16, 14), 1.00e-08)
for i, c in enumerate (SCALER_COL):
    D_ZERO[:,:,[i]] = SCALER.groupTransform(c,D_ZERO[:,:,[i]].reshape(-1,1)
                                           ,QUOTE_COL = QUOTE_COL
                                           ,VEGA_COL = VEGA_COL
                                           ,VOLUME_COL = VOLUME_COL
                                           ).reshape(-1, 16, 1)
a=pd.DataFrame(  
    D_ZERO.reshape(16, 14)
,columns=SCALER_COL)
a[:1]

In [ ]:
====================== CheckData =======================

In [ ]:
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
for partdate in PartitionDate[:1] :
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
        DATA_ORIGINAL = f[partdate][:]
    data_shape = DATA.shape
    print(f"CONFIG['transform'] : {CONFIG['transform']}")
    for i, c in enumerate (SCALER_COL):
        DATA[:,:,[i]] = SCALER.groupTransform(c,DATA[:,:,[i]].reshape(-1,1)
                                               ,QUOTE_COL = QUOTE_COL
                                               ,VEGA_COL = VEGA_COL
                                               ,VOLUME_COL = VOLUME_COL
                                               ).reshape(-1, 16, 1)


In [ ]:
col_x = [SCALER_COL[i] for i in select_x]
col_y = [SCALER_COL[i] for i in select_y]
DATA.shape

In [ ]:
DATA[:,:,SCALER_QUOTE_COL_INDEX][0]

In [ ]:
DATA_ORIGINAL[:,:,SCALER_QUOTE_COL_INDEX][0]

In [ ]:
for i in SCALER_QUOTE_COL_INDEX:
    DATA[:,:,i] = SCALER()['QUOTE'].inverse_transform(DATA[:,:,i] )
DATA[:,:,SCALER_QUOTE_COL_INDEX] [0]

In [ ]:
======================== predict =========================

In [ ]:
PartitionDate = [ d[:-3] for d in  os.listdir(H5_PATH)]
random.shuffle(PartitionDate)
for partdate in PartitionDate[:1] :
    DATA = []
    with h5py.File(H5_PATH+partdate+".h5", 'r') as f:
        DATA = f[partdate][:]
        DATA_ORIGINAL = f[partdate][:]
    data_shape = DATA.shape
    print(f"CONFIG['transform'] : {CONFIG['transform']}")
    for i, c in enumerate (SCALER_COL):
        DATA[:,:,[i]] = SCALER.groupTransform(c,DATA[:,:,[i]].reshape(-1,1)
                                               ,QUOTE_COL = QUOTE_COL
                                               ,VEGA_COL = VEGA_COL
                                               ,VOLUME_COL = VOLUME_COL
                                               ).reshape(-1, 16, 1)


In [ ]:
col_x = [SCALER_COL[i] for i in select_x]
col_y = [SCALER_COL[i] for i in select_y]

In [ ]:
X = DATA[:, :, select_x][:]
Y_real = DATA[:, :, select_y][:]
#===========
# X = x_train
# Y_real = y_train
# #===========
dfX = pd.DataFrame(
    X[:1].reshape(16, len(select_x)), 
    columns=col_x)
#print(dfX)

In [ ]:
genVal = model.generator(X) 
disVal = model.discriminator(genVal, training=False)
disVal_real = model.discriminator(Y_real, training=False)

In [ ]:
tf.reduce_mean(disVal) - tf.reduce_mean(disVal_real)

In [ ]:
genVal = genVal.numpy()
for i in range( len(SCALER_QUOTE_COL_INDEX) ):
    genVal[:,:,i] = SCALER()['QUOTE'].inverse_transform(genVal[:,:,i] )

In [ ]:
genVal[1]

In [ ]:
for i in range( len(SCALER_QUOTE_COL_INDEX) ):
    Y_real[:,:,i] = SCALER()['QUOTE'].inverse_transform(Y_real[:,:,i] )

In [ ]:
Y_real[1]

In [ ]:
6.98100000e+01

In [ ]:
1.00000000e-02

In [ ]:
========================== _LOSS_ANAYSIS ================================

In [ ]:
-tf.reduce_mean(disVal)

In [ ]:
data_dict = {
            [SCALER_COL[i] for i in select_y][i]: genVal[:, :, i]
            for i in range(len([SCALER_COL[i] for i in select_y]))
        }

In [ ]:
additional_con_loss_c_1 = tf.reduce_mean(
            tf.maximum(data_dict["C_ASK"] - data_dict["C_BID"], 0.0)
        )
additional_con_loss_c_1

In [ ]:
===

In [ ]:
additional_con_loss_c_2 = []
for c, v in [("C_BID", -4.999975e-14), ("C_ASK", -4.999975e-14)]:
    # tf mark with zero
    mask = tf.greater(data_dict[c], v)
    filtered_tensor = tf.boolean_mask(data_dict[c], mask)
    filtered_tensor_roll = tf.roll(filtered_tensor, shift=-1, axis=0)
    difference = tf.maximum(
        filtered_tensor_roll[:-1] - filtered_tensor[:-1], 0.0
    )
    additional_con_loss_c_2.append(tf.reduce_sum(difference))
additional_con_loss_c_2 = tf.reduce_mean(additional_con_loss_c_2)
additional_con_loss_c_2

In [ ]:
additional_con_loss_p_2 = []
for p, v in [("P_BID", -4.999975e-14), ("P_ASK", -4.999975e-14)]:
    # tf mark with zero
    mask = tf.greater(data_dict[p], v)
    filtered_tensor = tf.boolean_mask(data_dict[p], mask)
    filtered_tensor_roll = tf.roll(filtered_tensor, shift=-1, axis=0)
    difference = tf.maximum(
        filtered_tensor[:-1] - filtered_tensor_roll[:-1], 0.0
    )
    additional_con_loss_p_2.append(tf.reduce_sum(difference))
additional_con_loss_p_2 = tf.reduce_mean(additional_con_loss_p_2)
additional_con_loss_p_2

In [ ]:
v

In [ ]:
data_dict[p][0]

In [ ]:
mask[0]

In [ ]:
 tf.greater(data_dict[p][0], -4.999975e-14)

In [ ]:
==============

In [ ]:
dfY = pd.DataFrame(
    genVal.numpy()[:1].reshape(16, len(select_y)), 
    columns=col_y)


In [ ]:
npgenVal = genVal.numpy()
for i in range(genVal.shape[-1]):
    npgenVal[:,:,i] = SCALER()['QUOTE'].inverse_transform(npgenVal[:,:,i] )

In [ ]:
select_y

In [ ]:
pd.DataFrame(npgenVal[0],columns=col_y)

In [ ]:
Y_invT = Y_real
for i in range(Y_invT.shape[-1]):
    Y_invT[:,:,i] = SCALER()['QUOTE'].inverse_transform(Y_real[:,:,i] )
pd.DataFrame(Y_invT[0],columns=col_y)

In [ ]:
pd.DataFrame(DATA_ORIGINAL[:,:,select_y][0],columns=col_y)

In [ ]:
SUMDF = pd.concat([dfX, dfY],axis=1)

In [ ]:
resultDF = pd.DataFrame([])
rm_col = []
for i in SCALER_COL:
    if i in SUMDF.columns:
         resultDF[i] = SUMDF[i]
    else:
        rm_col += [i]
        resultDF[i] = [1e-8]*16


In [ ]:
if CONFIG['transform'] :
    decode_transformed=Scaler.inverse_transform(
        resultDF
    )
    decode_transformed=pd.DataFrame(  
        decode_transformed
    ,columns=SCALER_COL).drop(columns=rm_col)
    decode_transformed.loc[decode_transformed['DTE'] < 1e-8] = 0
    decode_transformed[col_y] = decode_transformed[col_y].round(2)
decode_transformed

In [ ]:
========================

In [ ]:
tdVal = tf.constant( resultDF[['C_BID','C_ASK']].values )

In [ ]:
mask = tf.greater(tdVal[:,0], -0.502343)

In [ ]:
filtered_tensor = tf.boolean_mask(tdVal[:,0], mask)
filtered_tensor

In [ ]:
filtered_tensor_roll = tf.roll(filtered_tensor, shift=-1, axis=0)
filtered_tensor_roll

In [ ]:
difference = tf.maximum(
                filtered_tensor_roll[:-1] - filtered_tensor[:-1], 0.0
            )
difference

In [ ]:
======================= real data ==============================

In [ ]:
if CONFIG['transform'] :
    realdecode_transformed=Scaler.inverse_transform(
        DATA[0]
    )
    realdecode_transformed=pd.DataFrame(  
        realdecode_transformed
    ,columns=SCALER_COL).drop(columns=rm_col)
    realdecode_transformed.loc[realdecode_transformed['DTE'] <= 1e-8] = 0
    

In [ ]:
realdecode_transformed

In [ ]:
9.999994e-09

In [ ]:
======================= _compute_loss ==============================

In [ ]:
#generated_data = [c_bid, c_ask, c_volume, p_bid, p_ask, p_volume]
colList = ["c_bid", "c_ask", "c_volume", "p_bid", "p_ask", "p_volume"]
generated_data = decode_data[3:]
z_mean    = z_mean
z_log_var = log_var
Y_real    = DATA[:, :, 3:][:1]

In [ ]:
z_mean

In [ ]:
for  col,genData in zip(colList,generated_data):
    print( colList.index(col),col )

In [ ]:
subtract_genData = genData - tf.cast(tf.expand_dims(Y_real[:, :, colList.index(col)], axis=-1)
        , tf.float32) 

In [ ]:
reconstruction_values_total = []
reconstruction_values_total.append( tf.reduce_mean( tf.square(subtract_genData)   ) )

In [ ]:
log_var = tf.clip_by_value(log_var, -1.0, 1.0)
kl_loss = -0.5 * tf.reduce_sum(1 + log_var - tf.square(z_mean) - tf.exp(log_var), axis=-1)

In [ ]:
tf.reduce_mean(reconstruction_values_total + kl_loss)

In [ ]:
========== kiras vae origi

In [ ]:
Y_real[0][0]

In [ ]:
 tf.concat(decode_data, axis=-1).numpy()[0][0]

In [ ]:
Y_real[0] -  tf.concat(decode_data, axis=-1).numpy()

In [ ]:
features_loss = tf.reduce_mean(
    tf.reduce_sum(
        tf.keras.losses.mean_squared_error(Y_real, tf.concat(decode_data, axis=-1)),
        axis=(1),
    )
)

In [ ]:
features_loss

In [ ]:
# features_loss = tf.reduce_mean(
#     tf.reduce_sum(
#         keras.losses.categorical_crossentropy(features_real, features_gen),
#         axis=(1),
#     )
# )
# kl_loss = -0.5 * tf.reduce_sum(
#     1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), 1
# )
# kl_loss = tf.reduce_mean(kl_loss)

# property_loss = tf.reduce_mean(
#     keras.losses.binary_crossentropy(qed_true, qed_pred)
# )

# graph_loss = self._gradient_penalty(graph_real, graph_generated)

# return kl_loss + property_loss + graph_loss + adjacency_loss + features_loss

In [ ]:
======================= inverse_transform ========================

In [ ]:
#add 0
decode_data = [tf.zeros([1, 32, 1])]*3 + decode_data

In [ ]:
invert_decode = Scaler.inverse_transform(
    np.array([d.numpy().reshape(-1) for d in decode_data]).transpose()
    ) 

In [ ]:
pd.DataFrame(
    invert_decode[:,3:], 
    columns=SCALER_COL[3:])

In [ ]:
=====================================================================

In [ ]:
DATA.reshape(-1,data_shape[-1])

In [ ]:
data_shape

In [ ]:
import numpy as np
matrix = np.array([
[1,2,3,],
[4,5,6],
[7,8,9]
])

In [ ]:
matrix = tf.convert_to_tensor(matrix)
    
# Shift matrix to compare each row with the next row
matrix_next = tf.roll(matrix, shift=-1, axis=0)

# Ignore the last row for comparison as it rolls over to the first row
matrix = matrix[:-1]
matrix_next = matrix_next[:-1]

# Compute the difference between each row and the next row
difference = matrix_next - matrix

In [ ]:
matrix_next

In [ ]:
matrix

In [ ]:
difference

In [ ]:
import json


In [ ]:
json.dumps( """ WHERE 
Count_Date < LEFT( DATEADD( month,2,CAST('$timestamp' AS date) ),7)+'-01')
AND Count_Date > LEFT( DATEADD( year,-2,CAST('$timestamp' AS date) ),4)+'-01-01' 
""")